In [1]:
# import flask modules
import numpy as np
from flask import Flask, request, jsonify

# import numpy and tensorflow modules
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# load the classifier model's h5 file
classifier = load_model("../notebooks/model.h5")

# create the flask app
app = Flask(__name__)

# routes HTTP request for '/predict' to predict_density() function
@app.route('/predict_density', methods= ['GET', 'POST'])
def predict_density():
    
    # verifies if the request method is GET
    if (request.method == 'GET'):
         # save the uploaded image into this directory
        file = request.files["file"]
        file.save(file.filename)
        
        # function call to predict the level of people within the area
        prediction = predict(file.filename, 224, 224)
        
        # make 'prediction' in JSON format
        prediction = jsonify(prediction)
        
        # returns JSON to the client
        return prediction
    else:
        return "Request method not valid"

def predict(image_file, img_width, img_height):
    # reformats image for Numpy Array conversion
    test_image = image.load_img(str(image_file), target_size= (img_width, img_height))
    
    # converts the image to an array
    image_matrix = image.img_to_array(test_image)
    
    # expands the shape of the array
    image_matrix = np.expand_dims(image_matrix, axis= 0)
    
    # classifies the level of people within the image
    result = classifier.predict(image_matrix)
    
    # returns best category classification
    categories = {0: "High", 1: "Low", 2: "Medium", 3: "Zero"}
    return get_predicted_population_density(result, categories)

def get_predicted_population_density(predictions, categories):
    current_category = 0
    best_category = 0

    # receive the category with the highest predicted possibility
    while(current_category < len(predictions[0])):
        if(predictions[0][current_category] > predictions[0][best_category]):
            # update the best category
            best_category = current_category
        current_category += 1
        
    return categories[best_category]

app.run(host= "127.0.0.1", port= "5000")

W1102 22:26:00.539215 139643429975872 deprecation.py:506] From /home/pravat/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1102 22:26:00.540187 139643429975872 deprecation.py:506] From /home/pravat/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1102 22:26:00.540790 139643429975872 deprecation.py:506] From /home/pravat/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tenso

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I1102 22:26:01.348944 139643429975872 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
